In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_pickle('./pickleJar/df_main.pkl')

In [3]:
df.head()

,id,etopo2,lat,rrs411,rrs443,rrs489,rrs510,rrs555,rrs670,chlor_a,is_hplc
0,1565,0.0,38.4279,0.001204,0.001686,0.003293,0.004036,0.007479,0.003465,38.19,False
1,1566,0.0,38.3680,0.001062,0.001384,0.002173,0.002499,0.004152,0.001695,35.01,False
2,1567,1.0,38.3074,0.000971,0.001185,0.001843,0.002288,0.004246,0.001612,26.91,False
3,1568,3.0,38.6367,0.001472,0.001741,0.002877,0.003664,0.006982,0.003234,47.96,False
4,1559,1.0,38.3047,0.000905,0.001022,0.001506,0.001903,0.002801,0.001791,23.55,False


In [4]:
df.describe()

,id,etopo2,lat,rrs411,rrs443,rrs489,rrs510,rrs555,rrs670,chlor_a
count,4459.000000,4459.000000,4459.000000,4293.000000,4456.000000,4422.000000,3435.000000,3255.000000,1598.000000,4127.000000
mean,4377.381251,1312.346715,1.868658,0.004881,0.004652,0.004590,0.004130,0.003256,0.001557,2.680228
std,2298.272102,1766.435289,44.765125,0.003447,0.003002,0.002768,0.003130,0.003536,0.002387,5.758436
min,6.000000,0.000000,-77.035600,0.000051,0.000190,0.000284,0.000261,0.000183,0.000000,0.012000
25%,2028.500000,18.000000,-61.299000,0.002509,0.002617,0.003051,0.002831,0.001588,0.000200,0.233325
50%,5039.000000,240.000000,27.093000,0.003984,0.003899,0.004153,0.003425,0.002071,0.000614,0.764000
75%,6271.500000,2789.500000,34.458500,0.006301,0.006076,0.005655,0.004242,0.003141,0.002000,2.150000
max,7831.000000,7978.000000,79.690000,0.030600,0.036769,0.063814,0.077740,0.046600,0.027700,77.864800


In [5]:
df.insert(df.shape[1]-2, 'CI',
          df.rrs555 - (df.rrs443
                       + (555-443) / (670-443)
                       * (df.rrs670 - df.rrs443)))

CI criteria for NOMAD (from [Hu*et al.*, 2012](http://dx.doi.org/10.1029/2011jc007395)):
* chl from hplc
* Rrs($\lambda$) > 0.0 sr^{-1}
* Chl > 0.0
* bottom depth > 30.0m
* latitude $\in$ [60$^\circ$S, 60$^\circ$N] 

In [22]:
# Applying above criteria
df.loc[(~df.is_hplc) | (df.rrs670<=0) |
       (df.chlor_a<=0) | (df.etopo2<=30) | (df.lat<-60) | 
       (df.lat>60), 'CI'] = np.NaN

In [23]:
df.describe()

,id,etopo2,lat,rrs411,rrs443,rrs489,rrs510,rrs555,rrs670,CI,chlor_a
count,4459.000000,4459.000000,4459.000000,4293.000000,4456.000000,4422.000000,3435.000000,3255.000000,1598.000000,145.000000,4127.000000
mean,4377.381251,1312.346715,1.868658,0.004881,0.004652,0.004590,0.004130,0.003256,0.001557,0.000015,2.680228
std,2298.272102,1766.435289,44.765125,0.003447,0.003002,0.002768,0.003130,0.003536,0.002387,0.002449,5.758436
min,6.000000,0.000000,-77.035600,0.000051,0.000190,0.000284,0.000261,0.000183,0.000000,-0.006578,0.012000
25%,2028.500000,18.000000,-61.299000,0.002509,0.002617,0.003051,0.002831,0.001588,0.000200,-0.001521,0.233325
50%,5039.000000,240.000000,27.093000,0.003984,0.003899,0.004153,0.003425,0.002071,0.000614,0.000687,0.764000
75%,6271.500000,2789.500000,34.458500,0.006301,0.006076,0.005655,0.004242,0.003141,0.002000,0.001483,2.150000
max,7831.000000,7978.000000,79.690000,0.030600,0.036769,0.063814,0.077740,0.046600,0.027700,0.006631,77.864800


In [24]:
ci_thresh = -0.0005
df.insert(df.shape[1]-2, 'within_CI_thresh',
          df.CI<ci_thresh)

In [25]:
df.within_CI_thresh = df.within_CI_thresh.astype('i')

In [26]:
df.head()

,id,etopo2,lat,rrs411,rrs443,rrs489,rrs510,rrs555,rrs670,CI,within_CI_thresh,chlor_a,is_hplc
0,1565,0.0,38.4279,0.001204,0.001686,0.003293,0.004036,0.007479,0.003465,NaN,0,38.19,False
1,1566,0.0,38.3680,0.001062,0.001384,0.002173,0.002499,0.004152,0.001695,NaN,0,35.01,False
2,1567,1.0,38.3074,0.000971,0.001185,0.001843,0.002288,0.004246,0.001612,NaN,0,26.91,False
3,1568,3.0,38.6367,0.001472,0.001741,0.002877,0.003664,0.006982,0.003234,NaN,0,47.96,False
4,1559,1.0,38.3047,0.000905,0.001022,0.001506,0.001903,0.002801,0.001791,NaN,0,23.55,False


In [27]:
df.within_CI_thresh.value_counts()

0    4410
1      49
Name: within_CI_thresh, dtype: int64

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Data columns (total 13 columns):
id                  4459 non-null int64
etopo2              4459 non-null float64
lat                 4459 non-null float64
rrs411              4293 non-null float64
rrs443              4456 non-null float64
rrs489              4422 non-null float64
rrs510              3435 non-null float64
rrs555              3255 non-null float64
rrs670              1598 non-null float64
CI                  145 non-null float64
within_CI_thresh    4459 non-null int32
chlor_a             4127 non-null float64
is_hplc             4459 non-null bool
dtypes: bool(1), float64(10), int32(1), int64(1)
memory usage: 405.0 KB


In [12]:
df.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1016 entries, 0 to 4458
Data columns (total 11 columns):
id                  1016 non-null int64
rrs411              1016 non-null float64
rrs443              1016 non-null float64
rrs489              1016 non-null float64
rrs510              1016 non-null float64
rrs555              1016 non-null float64
rrs670              1016 non-null float64
CI                  1016 non-null float64
within_CI_thresh    1016 non-null int32
chlor_a             1016 non-null float64
is_hplc             1016 non-null bool
dtypes: bool(1), float64(8), int32(1), int64(1)
memory usage: 84.3 KB


In [13]:
def insert_MBR(df_):
    insert_loc = df_.columns.get_loc('chlor_a')
    blue_cols = ['rrs443', 'rrs489', 'rrs510']
    df_.insert(insert_loc, 'MaxBlue', df_[blue_cols].max(axis=1))
    insert_loc += 1
    df_.insert(insert_loc, 'MaxBlueBand', df_[blue_cols].idxmax(axis=1).str.capitalize())
    insert_loc += 1
    df_.insert(insert_loc,'MaxBlueBandIdx',
               pd.Categorical(df_.MaxBlueBand, categories=[bluecol.capitalize() for bluecol in blue_cols]).codes)
    insert_loc += 1
    df_.insert(insert_loc, 'mxBlue2Gr', df_.MaxBlue / df_.rrs555)
    
    
def log_transform_for_chl_model(df_, merge=False):
    columns_to_log_transform = df_.filter(regex='(rrs|chl|mxBlue2)',
                                             axis=1).columns.tolist()
    if merge:
        for col in columns_to_log_transform:
            df_.insert(df_.columns.get_loc(col)+1,
                      f'log_{col}', np.log10(df.loc[:, col]))
        return None
    else:
        # log_transforms
        df_log = pd.DataFrame(np.log10(df_.loc[:,
                                               columns_to_log_transform
                                              ].values),
                              columns=[f'log_{col}' 
                                       for col in columns_to_log_transform])
        return df_log

In [14]:
insert_MBR(df)

In [15]:
log_transform_for_chl_model(df, merge=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Data columns (total 23 columns):
id                  4459 non-null int64
rrs411              4293 non-null float64
log_rrs411          4293 non-null float64
rrs443              4456 non-null float64
log_rrs443          4456 non-null float64
rrs489              4422 non-null float64
log_rrs489          4422 non-null float64
rrs510              3435 non-null float64
log_rrs510          3435 non-null float64
rrs555              3255 non-null float64
log_rrs555          3255 non-null float64
rrs670              1515 non-null float64
log_rrs670          1515 non-null float64
CI                  1127 non-null float64
within_CI_thresh    4459 non-null int32
MaxBlue             4459 non-null float64
MaxBlueBand         4459 non-null object
MaxBlueBandIdx      4459 non-null int8
mxBlue2Gr           3255 non-null float64
log_mxBlue2Gr       3255 non-null float64
chlor_a             4127 non-null float64
log_chlor_a        